# Introduction

**Overview:** Brief description of the problem, the dataset, and the main objectives of the project.

# Setup 

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

## Environment Variables 
**Note**: Setting environment variables is optional, but it is recommended if you store sensitive information (such as API keys or database credentials) in a `.env` file. Using environment variables helps keep such information secure and separate from your codebase.

In [ ]:
# Imports
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API key from .env 
api_key = os.getenv("API_KEY")

# Get SQL database credentials from .env
sql_username = os.getenv("SQL_USERNAME")
sql_password = os.getenv("SQL_PASSWORD")

# Data Loading

## csv 

In [ ]:
# Load data from a csv file into a Pandas DataFrame
df = pd.read_csv("your_csv_file_here.csv")

## MySQL

In [ ]:
# Imports 
from sqlalchemy import create_engine

# Database info
mysql_host = "localhost"  # Default hostname for a MySQL server running locally
mysql_port = 3306  # Default port for MySQL
mysql_database_name = "your_mysql_database_name_here"
mysql_table_name = "your_mysql_table_name_here"

# Create an SQLAlchemy engine for interacting with the MySQL database
engine = create_engine(f"mysql+mysqlconnector://{sql_username}:{sql_password}@{mysql_host}:{mysql_port}/{mysql_database_name}")

# Load data from MySQL database into a Pandas DataFrame
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT * FROM {mysql_table_name}", con=connection)

# Data Quality Checks

In [ ]:
# Show DataFrame info
df.info()

In [ ]:
# Show top five rows
df.head()

# Data Preprocessing

## Handling Duplicates

Duplicates based on all columns:

In [ ]:
# Diagnose duplicates 
df.duplicated().value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates().copy()

Duplicates based on specific columns, e.g. the ID column or a combination of columns:

In [ ]:
# Diagnose duplicates
df.duplicated(subset=["column_1", "column_2", "column_3"]).value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates(subset=["column_1", "column_2", "column_3"]).copy()

## Handling Incorrect Data Types

In [ ]:
# Convert column from str to int
df["int_column"] = df["str_column"].astype("Int32")

In [ ]:
# Convert column from str to datetime
df["datetime_column"] = pd.to_datetime(df["str_column"])

## Feature Extraction

### Categorical Feature from String Column

In [ ]:
# Function to extract a category from a string   
def extract_category_from_string(string):
    # Map categories to their corresponding list of keywords
    category_keywords_map = {
        "Category 1": ["Keyword 1", "Keyword 2", "Keyword 3"],
        "Category 2": ["Keyword 4", "Keyword 5", "Keyword 6"],
        "Category 3": ["Keyword 7", "Keyword 8", "Keyword 9"]
    }

    # Loop through each category and its associated keywords 
    for category, keywords in category_keywords_map.items():
        # Check if any keyword is present in the string
        if any(keyword in string for keyword in keywords):
            return category  # Return the category corresponding to the keyword
    return np.nan  # Return a missing value if no keyword matches

# Apply function on an existing string column to create a new categorical feature column
df["categorical_feature"] = df["str_column"].apply(extract_category_from_string)

# Show category frequencies
print(df["categorical_feature"].value_counts())

### Numerical Feature from String Column

In [ ]:
# Imports
import re

# Function to extract the first number in a string 
def extract_number_from_string(string):
    first_number = re.search(r"\b-?\d+([.,]\d+)?\b", string)  # searches for first integer or float (positive or negative; decimal separator "." or ",")
    if first_number:
        return float(first_number.group().replace(",", "."))  # Replace "," with "." as decimal separator  
    else:
        return np.nan  # Return a missing value if no number in string

# Apply function on an existing string column to create a new numerical feature column
df["numerical_feature"] = df["str_column"].apply(extract_number_from_string)

# Show descriptive statistics of numerical feature
df["numerical_feature"].describe()

### Boolean Feature from String Column

In [ ]:
# List of keywords to determine if the feature is present or absent
keywords = ["keyword 1", "keyword 2", "keyword 3"]

# Extract boolean feature column: True if any keyword is found in the string column
df["boolean_feature"] = df["str_column"].apply(lambda x: any(keyword.lower() in x.lower() for keyword in keywords))

# Show frequencies of boolean feature
print(df["boolean_feature"].value_counts())

## Handling Missing Values

### Imputation

Imputation for numerical column:

In [ ]:
# Descriptive statistics of numerical column
df["numerical_column"].describe()

In [ ]:
# Impute missing values with the median
median = df["numerical_column"].median()
df["numerical_column"] = df["numerical_column"].fillna(median)

Imputation for categorical column:

In [ ]:
# Frequencies of categorical column
df["categorical_column"].value_counts()

In [ ]:
# Impute missing values with the mode 
mode = df["categorical_column"].mode()[0]
df["categorical_column"] = df["categorical_column"].fillna(mode)

### Deletion

In [ ]:
# Delete rows where any column has a missing value 
df.dropna(inplace=True)

In [ ]:
# Delete rows where either column_1 or column_2 has a missing value 
df.dropna(subset=["column_1", "column_2"], how="any", inplace=True)

## Handling Outliers

### Remove with 3SD Method  
Remove univariate outliers from a numerical column by applying the 3 standard deviation (SD) rule. Specifically, a data point is considered an outlier if it falls more than 3 standard deviations above or below the mean of the column.  

In [ ]:
# Create a custom transformer class to remove outliers using the 3SD method
class OutlierRemover3SD(BaseEstimator, TransformerMixin):
    def fit(self, df, numerical_column):
        # Calculate mean, standard deviation, and cutoff values of the numerical column
        self.mean_ = df[numerical_column].mean()
        self.sd_ = df[numerical_column].std()
        self.lower_cutoff_ = self.mean_ - 3 * self.sd_
        self.upper_cutoff_ = self.mean_ + 3 * self.sd_

        # Create a mask for filtering outliers
        self.mask_ = (df[numerical_column] >= self.lower_cutoff_) & (df[numerical_column] <= self.upper_cutoff_)

        # Show cutoff values and number of outliers
        print(f"Lower cutoff for {numerical_column}: {self.lower_cutoff_}")
        print(f"Upper cutoff for {numerical_column}: {self.upper_cutoff_}")
        print(f"{df.shape[0] - df[self.mask_].shape[0]} outliers removed based on {numerical_column} using the 3SD method.")
  
        return self

    def transform(self, df):
        # Remove outliers based on the mask 
        return df[self.mask_]

    def fit_transform(self, df, numerical_column):
        # Perform both fit and transform 
        return self.fit(df, numerical_column).transform(df)

In [ ]:
# Initialize outlier remover 
outlier_remover_3sd = OutlierRemover3SD()

# Remove outliers
df = outlier_remover_3sd.fit_transform(df, "numerical_column")

### Remove with 1.5 IQR Method  
Remove univariate outliers from a numerical column using the 1.5 interquartile range (IQR) rule. Specifically, a data point is considered an outlier if it falls more than 1.5 interquartile ranges above the third quartile (Q3) or below the first quartile (Q1) of the column.  

In [ ]:
# Create a custom transformer class to remove outliers using the 1.5 IQR method
class OutlierRemoverIQR(BaseEstimator, TransformerMixin):
    def fit(self, df, numerical_column):
        # Calculate quartiles, IQR and cutoff values 
        Q1 = df[numerical_column].quantile(0.25)
        Q3 = df[numerical_column].quantile(0.75)
        IQR = Q3 - Q1
        self.lower_cutoff_ = Q1 - 1.5 * IQR
        self.upper_cutoff_ = Q3 + 1.5 * IQR

        # Create a mask for filtering outliers
        self.mask_ = (df[numerical_column] >= self.lower_cutoff_) & (df[numerical_column] <= self.upper_cutoff_)
  
        # Show cutoff values and number of outliers
        print(f"Lower cutoff for {numerical_column}: {self.lower_cutoff_}")
        print(f"Upper cutoff for {numerical_column}: {self.upper_cutoff_}")
        print(f"{df.shape[0] - df[self.mask_].shape[0]} outliers removed based on {numerical_column} using the 1.5 IQR method.")
        
        return self

    def transform(self, df):
        # Remove outliers based on the mask 
        return df[self.mask_]

    def fit_transform(self, df, numerical_column):
        # Perform both fit and transform
        return self.fit(df, numerical_column).transform(df)

In [ ]:
# Initialize outlier remover 
outlier_remover_iqr = OutlierRemoverIQR()

# Remove outliers
df = outlier_remover_iqr.fit_transform(df, "numerical_column")

# Exploratory Data Analysis (EDA)

## Defining Column Types  
Define numerical, categorical and boolean columns for downstream tasks like exploratory data analysis and machine learning.

In [ ]:
# Show columns and their pandas data types
df.info()

**Option 1: Manually**  
Use this approach for small datasets or when you have specific requirements and need explicit control.

In [ ]:
# Define column types manually
numerical_columns = ["numerical_column_1", "numerical_column_2", "numerical_column_3"]
categorical_columns = ["categorical_column_1", "categorical_column_2", "categorical_column_3"]
boolean_columns = ["boolean_column_1", "boolean_column_2", "boolean_column_3"]

**Option 2: Programmatically**  
Use this approach for large datasets or when you want to automate the process.

In [ ]:
# Define column types programmatically based on pandas data types
numerical_columns = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
boolean_columns = df.select_dtypes(include=["bool"]).columns.tolist()

## Descriptive Statistics

In [ ]:
# Table with descriptive statistics of all numerical columns
df[numerical_columns].describe().transpose()

## Numerical Target Variable

### Histogram

In [ ]:
# Create histogram of numerical target variable
sns.histplot(df["numerical_target"])

# Add title and axes labels 
plt.title("Distribution of numerical_target")
plt.xlabel("numerical_target")
plt.ylabel("Frequency")

# Show the plot
plt.show()

### Correlations

In [ ]:
# Correlations between the numerical target variable and each numerical and boolean column
combined_columns = numerical_columns + boolean_columns
df[combined_columns].corr()["numerical_target"].sort_values(ascending=False)

### Scatterplots 
Scatterplot matrix that shows scatterplots between the numerical target variable and each numerical column.   
Example code for 9 scatterplots in a 3x3 matrix:

In [ ]:
# Set the figure size 
plt.figure(figsize=(12, 12))

# List of numerical columns excluding the target column
numerical_columns_without_target = [col for col in numerical_columns if col != "numerical_target"]

# Iterate over the numerical columns
for i, numerical_column in enumerate(numerical_columns_without_target):
    # Create a subplot in a 3x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(3, 3, i + 1)
    
    # Create a scatterplot between the current column and the numerical target
    sns.scatterplot(data=df, x=numerical_column, y="numerical_target")
    
    # Add title and axis labels 
    plt.title(f"numerical_target by {numerical_column}")
    plt.xlabel(f"{numerical_column}")
    plt.ylabel("numerical_target")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

### Numerical Target Variable by Category

Descriptive statistics:

In [ ]:
# Descriptive statistics of numerical target variable by categorical column
target_by_category = df["numerical_target"].groupby(df["categorical_column"])
target_by_category.describe()

Bar plot:

In [ ]:
# Store the median of the numerical target by category
median_target_by_category = target_by_category.median()

# Bar plot of median by category
sns.barplot(x=median_target_by_category.index, y=median_target_by_category.values, palette="colorblind")

# Add title and axes labels
plt.title("Median numerical_target by categorical_column")
plt.xlabel("Category")
plt.ylabel("numerical_target")

# Show the plot
plt.show()

Bar plot matrix that shows bar plots between the numerical target variable and each categorical column.  
Example code for 5 bar plots in a 2x3 matrix:

In [ ]:
# Set the figure size to 12x6 inches
plt.figure(figsize=(12, 6))

# Iterate over the categorical columns
for i, categorical_column in enumerate(categorical_columns):
    # Create a subplot in a 2x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(2, 3, i + 1)
    
    # Create a bar plot of median numerical_target by the current categorical column
    ax = sns.barplot(data=df, x=categorical_column, y="numerical_target", estimator=np.median, ci=None)
    
    # Add title and axes labels
    plt.title(f"Median numerical_target by {categorical_column}")
    plt.xlabel("Category")
    plt.ylabel("numerical_target")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

## Categorical Columns

### Frequencies

#### Single Column

Absolute and relative frequencies of a single categorical colum:

In [ ]:
# Calculate absolute and relative frequencies 
absolute_frequencies = df["categorical_column"].value_counts()
relative_frequencies = absolute_frequencies / absolute_frequencies.sum() * 100

# Show frequencies
print(f"Absolute frequencies:\n {absolute_frequencies}\n")
print(f"Relative frequencies:\n {relative_frequencies.round(1)}")

Bar plot of frequencies:

In [ ]:
# Bar plot
sns.barplot(x=absolute_frequencies.index, y=absolute_frequencies.values, palette="colorblind")

# Add title and axes labels 
plt.title("categorical_column")
plt.ylabel("Frequency")

# Rotate x-axis tick labels by 45 degrees
plt.xticks(rotation=45)

# Show the plot
plt.show()

#### Multiple Columns

Absolute and relative frequencies of multiple categorical colums:

In [ ]:
# Initialize dictionaries to store frequencies for multiple categorical columns
absolute_frequencies_dict = {}
relative_frequencies_dict = {}

# Iterate over the categorical columns
for categorical_column in categorical_columns:
    # Calculate absolute and relative frequencies 
    absolute_frequencies = df[categorical_column].value_counts()
    relative_frequencies = absolute_frequencies / absolute_frequencies.sum() * 100

    # Store frequencies
    absolute_frequencies_dict[categorical_column] = absolute_frequencies
    relative_frequencies_dict[categorical_column] = relative_frequencies

    # Show frequencies
    print(categorical_column.upper())
    print(f"Absolute frequencies:\n {absolute_frequencies}\n")
    print(f"Relative frequencies (%):\n {relative_frequencies.round(1)}")
    print("=" * 50 + "\n")

Bar plot matrix that shows frequencies for multiple categorical columns.  
Example code for 5 bar plots in a 2x3 matrix:

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 6))

# Iterate over the categorical columns
for i, categorical_column in enumerate(categorical_columns):
    # Create a subplot in a 2x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(2, 3, i + 1)
    
    # Calculate frequencies for the current column
    absolute_frequencies = df[categorical_column].value_counts()
    
    # Create bar plot for the current column
    sns.barplot(x=absolute_frequencies.index, y=absolute_frequencies.values, estimator=np.median, ci=None)
    
    # Add title and axes labels
    plt.title(categorical_column.title())
    plt.ylabel("Frequency")

    # Rotate x-axis tick labels by 45 degrees
    plt.xticks(rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

## Correlations

Correlation matrix:

In [ ]:
# Combine numerical and boolean columns for correlation analysis
combined_columns = numerical_columns + boolean_columns

# Calculate the correlation matrix 
correlation_matrix = df[combined_columns].corr()

# Round correlations to 2 decimals
correlation_matrix = round(correlation_matrix, 2)

# Create an upper triangle mask (k=1 excludes the diagonal)
mask = np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)

# Set upper triangle to NaN to avoid redundant information
correlation_matrix[mask] = np.nan

# Show correlation matrix
correlation_matrix

Correlation heatmap:

In [ ]:
# Set the figure size
plt.figure(figsize=(8, 6))

# Create heatmap
sns.heatmap(
    correlation_matrix, 
    cmap="viridis",  # Color map choice
    annot=True,  # Show numbers
    linewidth=0.5  # Thin white lines between cells
)

# Add title
plt.title("Correlation Heatmap")

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

Save correlation heatmap as an image:

In [ ]:
# Imports
import os

# Create "images" directory if it doesn't exist
os.makedirs("images", exist_ok=True)

# Save the heatmap as a .png image
try:
    # Construct full file path
    image_path = os.path.join("images", "correlation_heatmap.png") 
    
    # Save the heatmap  
    plt.savefig(
        image_path, 
        bbox_inches="tight",  # removes unnecessary whitespace
        dpi=300  # higher image quality
    )
    print(f"Correlation heatmap saved successfully to {image_path}")

except Exception as e:
    print(f"Error saving correlation heatmap: {e}")

# Train-Validation-Test Split

In [ ]:
# Split the data into X features and y target
X = df.drop("target", axis=1)
y = df["target"]

In [ ]:
# Split the data into training and temporary sets (70% train, 30% temporary)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets (50% each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Delete the temporary data to free up memory
del X_temp, y_temp

Note: This accomplishes a 70% training, 15% validation and 15% test set size.

# Regression Task  
For a regression problem, where the goal is to predict a numerical target variable.

## Training Baseline Models  
Train five baseline models:
- Linear Regression
- Support Vector Regressor
- Random Forest Regressor
- Multi-Layer Perceptron Regressor
- XGBoost Regressor

The model performance will be evaluated using the following metrics:  
- Root Mean Squared Error (RMSE)
- Mean Absolute Percentage Error (MAPE)
- R-squared (R2)

### Helper Function for Residual Plots  
Creates two plots for predicted vs. actual target and residuals vs. actual target to evaluate model performance and identify potential issues in the model assumptions graphically.

In [ ]:
# Helper function to create residual plots
def plot_residuals(y, y_pred):
    # Calculate residuals
    residuals = [actual_value - predicted_value for actual_value, predicted_value in zip(y, y_pred)]

    # Create a 1x2 grid of subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), dpi=150)

    # Plot 1: Predicted vs. Actual Target
    axes[0].scatter(y, y_pred)
    axes[0].plot([min(y), max(y)], 
                 [min(y), max(y)], 
                 color="red", 
                 linestyle="--", 
                 label="Perfect Prediction")  # Add diagonal reference line
    axes[0].set_title("Predicted vs. Actual Target")
    axes[0].set_xlabel("Actual Target")
    axes[0].set_ylabel("Predicted Target")
    axes[0].grid(True)
    axes[0].legend() 

    # Plot 2: Residuals vs. Actual Target
    axes[1].scatter(y, residuals)
    axes[1].axhline(y=0, color="red", linestyle="--", label="Perfect Prediction")  # Add horizontal reference line
    axes[1].set_xlabel("Actual Target")
    axes[1].set_ylabel("Residuals")
    axes[1].set_title("Residuals vs. Actual Target")
    axes[1].grid(True)
    axes[1].legend() 

    # Adjust layout and display the plots
    plt.tight_layout()
    plt.show()

### Regression Pipeline

In [ ]:
# Define models with baseline configurations
models = [LinearRegression(), SVR(C=100), RandomForestRegressor(), MLPRegressor(max_iter=1000), XGBRegressor()]

# Create lists for storing the evaluation metrics (RMSE, MAPE, R2) of each model 
rmse_ls = []
mape_ls = []
r2_ls = []

# Loop through each model
for model in models:
    # Show model
    print("=" * 100)
    print(f"Model: {model}")

    # Scale numerical columns and encode categorical columns 
    column_transformer = ColumnTransformer(
        transformers=[
            ("scaler", StandardScaler(), numerical_columns),
            ("encoder", OneHotEncoder(drop=None, sparse_output=False), categorical_columns)
        ],
        remainder="passthrough"  # Include the boolean columns without transformation
    )

    # Create a pipeline
    pipeline = Pipeline(steps=[
        ("column_transformer", column_transformer),
        ("model", model)
    ])

    # Fit the pipeline on the training data
    pipeline.fit(X_train, y_train)
    
    # Predict on the validation data
    y_val_pred = pipeline.predict(X_val)

    # Calculate evaluation metrics: RMSE, MAPE, R2
    rmse = mean_squared_error(y_val, y_val_pred, squared=False)
    mape = mean_absolute_percentage_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)

    # Show evaluation metrics
    print(f"RMSE: {round(rmse, 2)}")
    print(f"MAPE: {round(mape, 2)}")
    print(f"R-squared (R²): {round(r2, 2)}")

    # Add evaluation metrics to their respective lists
    rmse_ls.append(rmse)
    mape_ls.append(mape)
    r2_ls.append(r2)

    # Create residual plots
    plot_residuals(y_val, y_val_pred)

# Classification Task  
For a classification problem, where the goal is to predict a categorical target variable.

# Summary

Data preprocessing steps:
- Handled duplicates (e.g., by removing duplicates based on the ID column).
- Handled incorrect data types (e.g., converted string columns to numerical columns where applicable).
- Extracted features (e.g., created categorical, numerical, or boolean features from string columns).
- Handled missing values (e.g., through imputation or deletion).

Next Steps:
- Split data into training, validation, and test sets (e.g., 70% train, 15% validation, 15% test).
- Scale numerical features (e.g., using StandardScaler).
- Encode categorical features (e.g., using One-Hot Encoding).
- Train baseline models (e.g., regression, random forest, XGBoost).
- Perform hyperparameter tuning (e.g., using GridSearchCV).
- Select final model based on performance evaluation (e.g., using accuracy, precision, recall, F1 score, R-squared, or RMSE).
- Save model weights (e.g., as a pickle file).
- Save preprocessed data (e.g., as a `.csv` file).